In [1]:
from keras.models import Model, model_from_json, load_model
from keras.layers import Input, LSTM, Dense
from keras.callbacks import ModelCheckpoint
import numpy as np
import keras
from tqdm import tqdm
import pandas as pd
import pickle
#from keras.utils.vis_utils import plot_model

c:\python3.5\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
keras.__version__

In [4]:
lines = open(''data/train/train_full.csv, encoding='utf-8').read().split('\n')
lines = lines[1:-1]

In [ ]:
train_filtered = total_train[total_train.apply(lambda x: not x['classtag'].endswith(('-','+','0')), axis=1)]

In [5]:
eng_sentences = []
fra_sentences = []
eng_chars = set()
fra_chars = set()
nb_samples = len(lines)

In [6]:
len(lines)

560640

In [6]:
# Process english and french sentences
for line in tqdm(range(nb_samples)):
    try:
        lemma,gender,formtag,classtag,form = lines[line].split('\t')
        eng_line = lemma+formtag
        #print(eng_line)
        # Append '\t' for start of the sentence and '\n' to signify end of the sentence
        fra_line = '\t' + form + '\n'
        #print(fra_line)
        eng_sentences.append(eng_line)
        fra_sentences.append(fra_line)

        for ch in eng_line:
            if (ch not in eng_chars):
                eng_chars.add(ch)

        for ch in fra_line:
            if (ch not in fra_chars):
                fra_chars.add(ch)
    except Exception as e:
        print(line, lines[line], e)

100%|████████████████████████████████████████| 422304/422304 [00:03<00:00, 127387.95it/s]


In [7]:
fra_chars = sorted(list(fra_chars))
eng_chars = sorted(list(eng_chars))

In [8]:
' '.join(eng_chars)

'- 1 2 A D G I L N а б в г д е ж з и й к л м н о п р с т у ф х ц ч ш щ ъ ы ь э ю я ё'

In [9]:
# dictionary to index each english character - key is index and value is english character
eng_index_to_char_dict = {}

# dictionary to get english character given its index - key is english character and value is index
eng_char_to_index_dict = {}

for k, v in enumerate(eng_chars):
    eng_index_to_char_dict[k] = v
    eng_char_to_index_dict[v] = k

In [10]:
# dictionary to index each french character - key is index and value is french character
fra_index_to_char_dict = {}

# dictionary to get french character given its index - key is french character and value is index
fra_char_to_index_dict = {}
for k, v in enumerate(fra_chars):
    fra_index_to_char_dict[k] = v
    fra_char_to_index_dict[v] = k

In [11]:
max_len_eng_sent = max([len(line) for line in eng_sentences])
max_len_fra_sent = max([len(line) for line in fra_sentences])

In [12]:
data = {}

In [13]:
data['eng_chars'] = eng_chars
data['fra_chars'] = fra_chars
data['max_len_eng_sent'] = max_len_eng_sent
data['max_len_fra_sent'] = max_len_fra_sent
data['eng_index_to_char_dict'] = eng_index_to_char_dict
data['eng_char_to_index_dict'] = eng_char_to_index_dict
data['fra_index_to_char_dict'] = fra_index_to_char_dict
data['fra_char_to_index_dict'] = fra_char_to_index_dict

In [15]:
#pickling data for inference model
pickle.dump(data, open( "data.p", "wb" ) )

In [2]:
data = pickle.load( open( "data.p", "rb" ) )


eng_chars = data['eng_chars']
fra_chars = data['fra_chars']
max_len_eng_sent = data['max_len_eng_sent']
max_len_fra_sent = data['max_len_fra_sent']
eng_index_to_char_dict = data['eng_index_to_char_dict']
eng_char_to_index_dict = data['eng_char_to_index_dict']
fra_index_to_char_dict = data['fra_index_to_char_dict']
fra_char_to_index_dict = data['fra_char_to_index_dict']

In [7]:
tokenized_eng_sentences = np.zeros(shape = (nb_samples,max_len_eng_sent,len(eng_chars)), dtype='float32')
tokenized_fra_sentences = np.zeros(shape = (nb_samples,max_len_fra_sent,len(fra_chars)), dtype='float32')
target_data = np.zeros((nb_samples, max_len_fra_sent, len(fra_chars)),dtype='float32')

In [13]:
tokenized_eng_sentences.shape

(560640, 26, 42)

In [11]:
tokenized_fra_sentences.shape

(560640, 28, 37)

In [10]:
# Vectorize the english and french sentences

for i in tqdm(range(nb_samples)):
    for k,ch in enumerate(eng_sentences[i]):
        tokenized_eng_sentences[i,k,eng_char_to_index_dict[ch]] = 1

    for k,ch in enumerate(fra_sentences[i]):
        tokenized_fra_sentences[i,k,fra_char_to_index_dict[ch]] = 1

        # decoder_target_data will be ahead by one timestep and will not include the start character.
        if k > 0:
            target_data[i,k-1,fra_char_to_index_dict[ch]] = 1


  0%|                                                                                       | 0/560640 [00:00<?, ?it/s]


IndexError: list index out of range

In [8]:
# Encoder model

encoder_input = Input(shape=(None,len(eng_chars)))
encoder_LSTM = LSTM(256,return_state = True)
encoder_outputs, encoder_h, encoder_c = encoder_LSTM (encoder_input)
encoder_states = [encoder_h, encoder_c]


In [9]:
encoder_input.shape

TensorShape([Dimension(None), Dimension(None), Dimension(42)])

In [ ]:
# Decoder model

decoder_input = Input(shape=(None,len(fra_chars)))
decoder_LSTM = LSTM(256,return_sequences=True, return_state = True)
decoder_out, _ , _ = decoder_LSTM(decoder_input, initial_state=encoder_states)
decoder_dense = Dense(len(fra_chars),activation='softmax')
decoder_out = decoder_dense (decoder_out)


In [ ]:
model = Model(inputs=[encoder_input, decoder_input],outputs=[decoder_out])
#plot_model(tmodel, to_file='tmodel.png', show_shapes=True)

In [ ]:
%%time
model = Model(inputs=[encoder_input, decoder_input],outputs=[decoder_out])

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

filepath="weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

model.fit(x=[tokenized_eng_sentences,tokenized_fra_sentences], 
          y=target_data,
          batch_size=64,
          epochs=50,
          validation_split=0.2,
          #callbacks=callbacks_list,
         verbose = 2)


In [ ]:
#model.save('model.h5')

### creating and saving inference models

In [ ]:
# Inference models for testing
# Encoder inference model
encoder_model_inf = Model(encoder_input, encoder_states)

In [ ]:
# Decoder inference model
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_input_states = [decoder_state_input_h, decoder_state_input_c]

decoder_out, decoder_h, decoder_c = decoder_LSTM(decoder_input, 
                                                 initial_state=decoder_input_states)

decoder_states = [decoder_h , decoder_c]

decoder_out = decoder_dense(decoder_out)

decoder_model_inf = Model(inputs=[decoder_input] + decoder_input_states,
                          outputs=[decoder_out] + decoder_states )

In [ ]:
encoder_model_inf.save('seq2seq/encoder_total.h5')
decoder_model_inf.save('seq2seq/decoder_total.h5')

In [ ]:
encoder_model_inf.save('seq2seq/encoder_total.h5')
decoder_model_inf.save('seq2seq/decoder_total.h5')

In [8]:
encoder_model_inf = load_model('retraining_models/encoder-1.h5')
decoder_model_inf = load_model('retraining_models/decoder-1.h5')

c:\python3.5\lib\site-packages\keras\models.py:255: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [10]:
encoder_model_inf = load_model('models/encoder_full_letters.h5')
decoder_model_inf = load_model('models/decoder_full_letters.h5')

c:\python3.5\lib\site-packages\keras\models.py:255: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [ ]:
del model
del encoder_model_inf
del decoder_model_inf

### predicting result

In [5]:
def decode_seq(inp_seq,encoder_model_inf,decoder_model_inf):
    
    # Initial states value is coming from the encoder 
    states_val = encoder_model_inf.predict(inp_seq)
    
    target_seq = np.zeros((1, 1, len(fra_chars)))
    target_seq[0, 0, fra_char_to_index_dict['\t']] = 1
    
    translated_sent = ''
    stop_condition = False
    
    prob = 1.0
    while not stop_condition:
        
        #decoder_out, decoder_h, decoder_c = decoder_model_inf.predict(x=[target_seq] + states_val)
        #predict_pr = decoder_model_inf.predict_proba(x=[target_seq] + states_val)
        decoder_out, decoder_h, decoder_c = decoder_model_inf.predict(x=[target_seq] + states_val)
        #print(decoder_out)
        max_val_index = np.argmax(decoder_out[0,-1,:])
        max_val = np.max(decoder_out[0,-1,:])
        prob *= max_val
        sampled_fra_char = fra_index_to_char_dict[max_val_index]
        translated_sent += sampled_fra_char
        print('{} == {}'.format(sampled_fra_char,max_val))
        
        if ( (sampled_fra_char == '\n') or (len(translated_sent) > max_len_fra_sent)) :
            stop_condition = True
        
        target_seq = np.zeros((1, 1, len(fra_chars)))
        target_seq[0, 0, max_val_index] = 1
        
        states_val = [decoder_h, decoder_c]
    
    prob = prob**(1/len(translated_sent))
    return translated_sent, prob

In [6]:
def tokenize(seq):
    tokenized_eng_sentence = np.zeros(shape = (1,max_len_eng_sent,len(eng_chars)), dtype='float32')
    for k,ch in enumerate(seq):
        tokenized_eng_sentence[0,k,eng_char_to_index_dict[ch]] = 1
        
    return tokenized_eng_sentence


In [11]:
inp = 'брат2G'
inp_seq = tokenize(inp)
translated_sent,prob = decode_seq(inp_seq,encoder_model_inf,decoder_model_inf)
prob

б == 0.9932345747947693
р == 1.0
а == 1.0
т == 1.0
ь == 0.9999997615814209
е == 0.9999508857727051
в == 1.0

 == 1.0


0.9991456441333508

In [6]:
word = 'огуречик'
d = pd.DataFrame(np.zeros((6, 2)),index=['Nom','Gen','Dat','Acc','Ins','Loc',],columns=['Singular','Plural'])
    tags = [[i,k,n + c] for i,n in enumerate(['1','2']) for k,c in enumerate(['N','G','D','A','I','L'])]
    for n,c,tag in tags:
        inp = word + tag
        inp_seq = tokenize(inp)
        translated_sent = decode_seq(inp_seq)
        d.iloc[c,n] = translated_sent.rstrip()

IndentationError: unexpected indent (<ipython-input-6-d498dbd4683d>, line 3)

In [29]:
prob**(1/len)

2.23606797749979